In [1]:
import pandas as pd

In [2]:
portfolio_input_file = "../Datasets/portfolio.csv"
profile_input_file = "../Datasets/profile.csv"
transcript_input_file = "../Datasets/transcript.csv"

In [3]:
portfolio_raw = pd.read_csv(portfolio_input_file, index_col=0)
profile_raw = pd.read_csv(profile_input_file, index_col=0)
transcript_raw = pd.read_csv(transcript_input_file, index_col=0)

#### Reading Raw Datasets

In [4]:
portfolio_df = portfolio_raw.copy()
portfolio_df.info()
portfolio_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 560.0+ bytes


,reward,channels,difficulty,duration,offer_type,id
0,10,"['email', 'mobile', 'social']",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"['web', 'email', 'mobile', 'social']",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"['web', 'email', 'mobile']",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"['web', 'email', 'mobile']",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"['web', 'email']",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [5]:
profile_df = profile_raw.copy()
profile_df.info()
profile_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 796.9+ KB


,gender,age,id,became_member_on,income
0,NaN,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,NaN,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,NaN,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [6]:
profile_df["became_member_on"] = pd.to_datetime(profile_df["became_member_on"], format="%Y%m%d")
profile_df.head()

,gender,age,id,became_member_on,income
0,NaN,118,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0
2,NaN,118,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0
4,NaN,118,a03223e636434f42ac4c3df47e8bac43,2017-08-04,NaN


In [7]:
transcript_df = transcript_raw.copy()
transcript_df.info()
transcript_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   person  306534 non-null  object
 1   event   306534 non-null  object
 2   value   306534 non-null  object
 3   time    306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 11.7+ MB


,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [8]:
transcript_df["event"].value_counts()

event
transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: count, dtype: int64

#### Offer Events

In [9]:
offer_completed_df = transcript_df.copy()[transcript_df["event"] == "offer completed"].reset_index(drop=True)
# Extract offer id
offer_completed_df["offer_id"] = [eval(value_dict).get("offer_id") for value_dict in offer_completed_df["value"]]
# Drop event and value columns
offer_completed_df = offer_completed_df.drop(["event", "value"], axis=1)
# Rename time column
offer_completed_df = offer_completed_df.rename(columns={"time":"completed_time"})

offer_completed_df.info()
offer_completed_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33579 entries, 0 to 33578
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   person          33579 non-null  object
 1   completed_time  33579 non-null  int64 
 2   offer_id        33579 non-null  object
dtypes: int64(1), object(2)
memory usage: 787.1+ KB


,person,completed_time,offer_id
0,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,2906b810c7d4411798c6938adc9daaa5
1,fe97aa22dd3e48c8b143116a8403dd52,0,fafdcd668e3743c1bb461111dcafc2a4
2,629fc02d56414d91bca360decdfa9288,0,9b98b8c7a33c4b65b9aebfe6a799e6d9
3,676506bad68e4161b9bbaffeb039626b,0,ae264e3637204a6fb9bb56bc8210ddfd
4,8f7dd3b2afe14c078eb4f6e6fe4ba97d,0,4d5c57ea9a6940dd891ad53e9dbe8da0


In [10]:
offer_received_df = transcript_df.copy()[transcript_df["event"] == "offer received"].reset_index(drop=True)
# Extract offer id
offer_received_df["offer_id"] = [eval(value_dict).get("offer id") for value_dict in offer_received_df["value"]]
# Drop event and value columns
offer_received_df = offer_received_df.drop(["event", "value"], axis=1)
# Rename time column
offer_received_df = offer_received_df.rename(columns={"time":"received_time"})

offer_received_df.info()
offer_received_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76277 entries, 0 to 76276
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   person         76277 non-null  object
 1   received_time  76277 non-null  int64 
 2   offer_id       76277 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.7+ MB


,person,received_time,offer_id
0,78afa995795e4d85b5d9ceeca43f5fef,0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,a03223e636434f42ac4c3df47e8bac43,0,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,e2127556f4f64592b11af22de27a7932,0,2906b810c7d4411798c6938adc9daaa5
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,fafdcd668e3743c1bb461111dcafc2a4
4,68617ca6246f4fbc85e91a2a49552598,0,4d5c57ea9a6940dd891ad53e9dbe8da0


In [11]:
offer_viewed_df = transcript_df.copy()[transcript_df["event"] == "offer viewed"].reset_index(drop=True)
# Extract offer id
offer_viewed_df["offer_id"] = [eval(value_dict).get("offer id") for value_dict in offer_viewed_df["value"]]
# Drop event and value columns
offer_viewed_df = offer_viewed_df.drop(["event", "value"], axis=1)
# Rename time column
offer_viewed_df = offer_viewed_df.rename(columns={"time":"viewed_time"})

offer_viewed_df.info()
offer_viewed_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57725 entries, 0 to 57724
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   person       57725 non-null  object
 1   viewed_time  57725 non-null  int64 
 2   offer_id     57725 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.3+ MB


,person,viewed_time,offer_id
0,389bc3fa690240e798340f5a15918d5c,0,f19421c1d4aa40978ebb69ca19b0e20d
1,d1ede868e29245ea91818a903fec04c6,0,5a8bc65990b245e5a138643cd4eb9837
2,102e9454054946fda62242d2e176fdce,0,4d5c57ea9a6940dd891ad53e9dbe8da0
3,02c083884c7d45b39cc68e1314fec56c,0,ae264e3637204a6fb9bb56bc8210ddfd
4,be8a5d1981a2458d90b255ddc7e0d174,0,5a8bc65990b245e5a138643cd4eb9837


#### Match offer viewed and offer completed to offer received entry

In [127]:
# Merge offer duration information
offer_received_df_merged = offer_received_df.merge(portfolio_df[["id", "duration"]], how="left", left_on="offer_id", right_on="id")
# Convert offer duration (given in days) to hours
offer_received_df_merged["offer_end_time"] = offer_received_df_merged["received_time"] + (offer_received_df_merged["duration"] * 24)
offer_received_df_merged = offer_received_df_merged.drop(columns=["duration"])

offer_received_df_merged.info()
offer_received_df_merged.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76277 entries, 0 to 76276
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   person          76277 non-null  object
 1   received_time   76277 non-null  int64 
 2   offer_id        76277 non-null  object
 3   id              76277 non-null  object
 4   offer_end_time  76277 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 2.9+ MB


,person,received_time,offer_id,id,offer_end_time
0,78afa995795e4d85b5d9ceeca43f5fef,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,9b98b8c7a33c4b65b9aebfe6a799e6d9,168
1,a03223e636434f42ac4c3df47e8bac43,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0b1e1539f2cc45b7b9fa7c272da2e1d7,240
2,e2127556f4f64592b11af22de27a7932,0,2906b810c7d4411798c6938adc9daaa5,2906b810c7d4411798c6938adc9daaa5,168
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,fafdcd668e3743c1bb461111dcafc2a4,fafdcd668e3743c1bb461111dcafc2a4,240
4,68617ca6246f4fbc85e91a2a49552598,0,4d5c57ea9a6940dd891ad53e9dbe8da0,4d5c57ea9a6940dd891ad53e9dbe8da0,120


In [13]:
# Sort by ascending time
offer_received_df_merged = offer_received_df_merged.sort_values(by=["received_time", "person", "offer_id"])
offer_viewed_df = offer_viewed_df.sort_values(by=["viewed_time", "person", "offer_id"])
offer_completed_df = offer_completed_df.sort_values(by=["completed_time", "person", "offer_id"])

In [14]:
from collections import defaultdict

# Intialise dictionary to store offer_viewed times with list as default value
offer_viewed_dict = defaultdict(list)
# Use (person_id, offer_id) as the key
# Append the offer_viewed timings to the list tagged to the key
for index, row in offer_viewed_df.iterrows():
    offer_viewed_dict[(row['person'], row['offer_id'])].append(row['viewed_time'])

# Intialise dictionary to store offer_completed times with list as default value
offer_completed_dict = defaultdict(list)
# Use (person_id, offer_id) as the key
# Append the offer_completed timings to the list tagged to the key
for index, row in offer_completed_df.iterrows():
    offer_completed_dict[(row['person'], row['offer_id'])].append(row['completed_time'])

# Initiaise dictionary to store offer records for each person
person_offer_metrics_dict = {}

# Extract offer_received information to the dictionary
for index, row in offer_received_df_merged.iterrows():
    person = row['person']
    offer_id = row['offer_id']
    received_time = row['received_time']
    offer_end_time = row["offer_end_time"]
    # Implement unqiue offer key as offer_id can appear more than once for the same person
    unique_offer_key = (offer_id, received_time, index) 
    
    # Add person as key in the dictionary
    if person not in person_offer_metrics_dict:
        person_offer_metrics_dict[person] = {}

    # Tag unique_offer_key to the person and fill in the offer received record in a sub-dictionary
    if unique_offer_key not in person_offer_metrics_dict[person]:
        person_offer_metrics_dict[person][unique_offer_key] = {
            'received_time': received_time,
            'offer_end_time': offer_end_time,
            'viewed_time': None,
            'completed_time': None,
        }
        
    # Match the offer_viewed time. None by default.
    if (person, offer_id) in offer_viewed_dict:
        # Enumerate list to obtain position of value in the list
        for i, view_time in enumerate(offer_viewed_dict[(person, offer_id)]):
            # Check if the offer was viewed after offer received and viewed within offer validity period
            if received_time <= view_time <= offer_end_time:
                # Update viewed_time
                person_offer_metrics_dict[person][unique_offer_key]['viewed_time'] = view_time
                # Remove matched offer_viewed time
                offer_viewed_dict[(person, offer_id)].pop(i)
                break

    # Match the offer_completed time. None by default.   
    if (person, offer_id) in offer_completed_dict:
        for i, complete_time in enumerate(offer_completed_dict[(person, offer_id)]):
            # Check if the offer was completed after offer received and completed within offer validity period
            if received_time <= complete_time <= offer_end_time:
                # Update completed_time
                person_offer_metrics_dict[person][unique_offer_key]['completed_time'] = complete_time
                # Remove matched offer_completed time
                offer_completed_dict[(person, offer_id)].pop(i)
                break

In [15]:
# Extract data from person_offer_metrics_dict into dataframe
offers_data = []

for person, offers in person_offer_metrics_dict.items():
    for offer_id, metrics in offers.items():
        offers_data.append({
            'person': person,
            'offer_id': offer_id[0],
            'received_time': metrics["received_time"],
            'viewed_time': metrics["viewed_time"],
            'completed_time': metrics["completed_time"],
            'offer_end_time': metrics["offer_end_time"],
            'unique_offer_id': offer_id,
        })

offers_df = pd.DataFrame(offers_data)

# Standardise numeric types to float
offers_df["received_time"] = offers_df["received_time"].astype(float)
offers_df["offer_end_time"] = offers_df["offer_end_time"].astype(float)

offers_df.info()
offers_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76277 entries, 0 to 76276
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   person           76277 non-null  object 
 1   offer_id         76277 non-null  object 
 2   received_time    76277 non-null  float64
 3   viewed_time      56567 non-null  float64
 4   completed_time   33579 non-null  float64
 5   offer_end_time   76277 non-null  float64
 6   unique_offer_id  76277 non-null  object 
dtypes: float64(4), object(3)
memory usage: 4.1+ MB


,person,offer_id,received_time,viewed_time,completed_time,offer_end_time,unique_offer_id
0,0011e0d4e6b944f998e987f904e8c1e5,3f207df678b143eea3cee63160fa8bed,0.0,6.0,NaN,96.0,"(3f207df678b143eea3cee63160fa8bed, 0, 53673)"
1,0011e0d4e6b944f998e987f904e8c1e5,2298d6c36e964ae4a3e7e9706d1fb8c2,168.0,186.0,252.0,336.0,"(2298d6c36e964ae4a3e7e9706d1fb8c2, 168, 47257)"
2,0011e0d4e6b944f998e987f904e8c1e5,5a8bc65990b245e5a138643cd4eb9837,336.0,354.0,NaN,408.0,"(5a8bc65990b245e5a138643cd4eb9837, 336, 71497)"
3,0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7,408.0,432.0,576.0,648.0,"(0b1e1539f2cc45b7b9fa7c272da2e1d7, 408, 11831)"
4,0011e0d4e6b944f998e987f904e8c1e5,9b98b8c7a33c4b65b9aebfe6a799e6d9,504.0,516.0,576.0,672.0,"(9b98b8c7a33c4b65b9aebfe6a799e6d9, 504, 5398)"


In [16]:
# Check if distribution of offer received time tallies with the original
offers_df["received_time"].describe() == offer_received_df_merged["received_time"].describe()

count     True
mean      True
std      False
min       True
25%       True
50%       True
75%       True
max       True
Name: received_time, dtype: bool

In [17]:
# Check the difference in std
offers_df["received_time"].describe()["std"] - offer_received_df_merged["received_time"].describe()["std"]

-1.304556462855544e-11

In [18]:
# Check if distribution of offer completed time tallies with the original
offers_df["completed_time"].describe() == offer_completed_df["completed_time"].describe()

count     True
mean      True
std      False
min       True
25%       True
50%       True
75%       True
max       True
Name: completed_time, dtype: bool

In [19]:
# Check the difference in std
offers_df["completed_time"].describe()["std"] - offer_completed_df["completed_time"].describe()["std"]

1.1652900866465643e-12

In [20]:
# Check if distribution of offer viewed time tallies with the original
offers_df["viewed_time"].describe() == offer_viewed_df["viewed_time"].describe()

count    False
mean     False
std      False
min       True
25%       True
50%       True
75%       True
max       True
Name: viewed_time, dtype: bool

In [21]:
offers_df["viewed_time"].describe()

count    56567.000000
mean       353.750314
std        199.139007
min          0.000000
25%        180.000000
50%        408.000000
75%        516.000000
max        714.000000
Name: viewed_time, dtype: float64

In [22]:
offer_viewed_df["viewed_time"].describe()

count    57725.000000
mean       354.290515
std        199.317684
min          0.000000
25%        180.000000
50%        408.000000
75%        516.000000
max        714.000000
Name: viewed_time, dtype: float64

- Lower count of viewed_time as view times that were recorded after the offer duration were ignored. Not considered a valid view.

In [23]:
# Add boolean columns to indicate if offer was viewed/completed. 1 indicates action has been taken. 
offers_df["offer_viewed"] = offers_df["viewed_time"].notna().astype(int)
offers_df["offer_completed"] = offers_df["completed_time"].notna().astype(int)

#### Merge offer information

In [24]:
# Extract offer channel information into new columns
channel_list_of_lists = [eval(channel_list) for channel_list in portfolio_df["channels"]]

num_channels_list = [len(channel_list) for channel_list in channel_list_of_lists]
used_email_list = [1 if "email" in channel_list else 0 for channel_list in channel_list_of_lists]
used_web_list = [1 if "web" in channel_list else 0 for channel_list in channel_list_of_lists]
used_social_list = [1 if "social" in channel_list else 0 for channel_list in channel_list_of_lists]
used_mobile_list = [1 if "mobile" in channel_list else 0 for channel_list in channel_list_of_lists]

portfolio_df_transformed = portfolio_df.copy()
portfolio_df_transformed["num_channels"] = num_channels_list
portfolio_df_transformed["used_email"] = used_email_list
portfolio_df_transformed["used_web"] = used_web_list
portfolio_df_transformed["used_social"] = used_social_list
portfolio_df_transformed["used_mobile"] = used_mobile_list

portfolio_df_transformed = portfolio_df_transformed.drop(columns=["channels"])

portfolio_df_transformed.info()
portfolio_df_transformed.head()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 0 to 9
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   reward        10 non-null     int64 
 1   difficulty    10 non-null     int64 
 2   duration      10 non-null     int64 
 3   offer_type    10 non-null     object
 4   id            10 non-null     object
 5   num_channels  10 non-null     int64 
 6   used_email    10 non-null     int64 
 7   used_web      10 non-null     int64 
 8   used_social   10 non-null     int64 
 9   used_mobile   10 non-null     int64 
dtypes: int64(8), object(2)
memory usage: 880.0+ bytes


,reward,difficulty,duration,offer_type,id,num_channels,used_email,used_web,used_social,used_mobile
0,10,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,3,1,0,1,1
1,10,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,4,1,1,1,1
2,0,0,4,informational,3f207df678b143eea3cee63160fa8bed,3,1,1,0,1
3,5,5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,3,1,1,0,1
4,5,20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,2,1,1,0,0


In [128]:
# Merge offer information with offers_df
offers_df = offers_df.merge(portfolio_df_transformed, how="left", left_on="offer_id", right_on="id")
offers_df = offers_df.drop(columns=["id"])

offers_df.info()
offers_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76277 entries, 0 to 76276
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   person           76277 non-null  object 
 1   offer_id         76277 non-null  object 
 2   received_time    76277 non-null  float64
 3   viewed_time      56567 non-null  float64
 4   completed_time   33579 non-null  float64
 5   offer_end_time   76277 non-null  float64
 6   unique_offer_id  76277 non-null  object 
 7   offer_viewed     76277 non-null  int32  
 8   offer_completed  76277 non-null  int32  
 9   reward_x         76277 non-null  int64  
 10  difficulty_x     76277 non-null  int64  
 11  duration_x       76277 non-null  int64  
 12  offer_type_x     76277 non-null  object 
 13  num_channels_x   76277 non-null  int64  
 14  used_email_x     76277 non-null  int64  
 15  used_web_x       76277 non-null  int64  
 16  used_social_x    76277 non-null  int64  
 17  used_mobile_

,person,offer_id,received_time,viewed_time,completed_time,offer_end_time,unique_offer_id,offer_viewed,offer_completed,reward_x,...,used_mobile_x,reward_y,difficulty_y,duration_y,offer_type_y,num_channels_y,used_email_y,used_web_y,used_social_y,used_mobile_y
0,0011e0d4e6b944f998e987f904e8c1e5,3f207df678b143eea3cee63160fa8bed,0.0,6.0,NaN,96.0,"(3f207df678b143eea3cee63160fa8bed, 0, 53673)",1,0,0,...,1,0,0,4,informational,3,1,1,0,1
1,003d66b6608740288d6cc97a6903f4f0,3f207df678b143eea3cee63160fa8bed,336.0,372.0,NaN,432.0,"(3f207df678b143eea3cee63160fa8bed, 336, 56211)",1,0,0,...,1,0,0,4,informational,3,1,1,0,1
2,0056df74b63b4298809f0b375a304cf4,3f207df678b143eea3cee63160fa8bed,504.0,528.0,NaN,600.0,"(3f207df678b143eea3cee63160fa8bed, 504, 59450)",1,0,0,...,1,0,0,4,informational,3,1,1,0,1
3,00715b6e55c3431cb56ff7307eb19675,3f207df678b143eea3cee63160fa8bed,408.0,NaN,NaN,504.0,"(3f207df678b143eea3cee63160fa8bed, 408, 57961)",0,0,0,...,1,0,0,4,informational,3,1,1,0,1
4,0082fd87c18f45f2be70dbcbb0fb8aad,3f207df678b143eea3cee63160fa8bed,168.0,168.0,NaN,264.0,"(3f207df678b143eea3cee63160fa8bed, 168, 54748)",1,0,0,...,1,0,0,4,informational,3,1,1,0,1


In [26]:
# Save to csv file
offers_df.to_csv("offers_df.csv")

#### Obtain Customer Profile

In [27]:
transactions_df = transcript_df.copy()[transcript_df["event"] == "transaction"].reset_index(drop=True)

# Extract amount
transactions_df["amount"] = [eval(value_dict).get("amount") for value_dict in transactions_df["value"]]
# Round to amount 2 dp
transactions_df["amount"] = round(transactions_df["amount"], 2)
# Drop event and value columns
transactions_df = transactions_df.drop(columns=["event", "value"])
# Indicate week based on time. Perform floor division. Start with week 1.
transactions_df["week"] = (transactions_df["time"] // (24*7)) + 1
transactions_df["week"] = transactions_df["week"].astype(str)

transactions_df.info()
transactions_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138953 entries, 0 to 138952
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   person  138953 non-null  object 
 1   time    138953 non-null  int64  
 2   amount  138953 non-null  float64
 3   week    138953 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 4.2+ MB


,person,time,amount,week
0,02c083884c7d45b39cc68e1314fec56c,0,0.83,1
1,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56,1
2,54890f68699049c2a04d415abc25e717,0,13.23,1
3,b2f1cd155b864803ad8334cdf13c4bd2,0,19.51,1
4,fe97aa22dd3e48c8b143116a8403dd52,0,18.97,1


In [90]:
# End of period taken as 714 hours

# Compute time between transactions
def get_average_time_between_transactions(time_list, end_of_period=714):
    if len(time_list) > 0:
        # If only 1 transaction occurred, find difference with end of period
        if len(time_list) == 1:
            time_diff_list = [end_of_period - time_list[0]]
        else:
            # Sort list in ascending time order
            time_list = sorted(time_list)
            # Find difference between consecutive transaction times
            time_diff_list = [time_list[i+1] - time_list[i] for i in range(len(time_list)-1)]
        # Find average difference (in hours)
        return round(sum(time_diff_list) / len(time_diff_list), 2)
    else:
        return None

# Compute Recency: hours since last transaction from end of period
def get_hours_since_last_transaction(time_list, end_of_period=714):
    if len(time_list) > 0:
        # Find latest transaction time
        latest_transaction_time = max(time_list)
        # Find hours since (compared to end of period) 
        return (end_of_period - latest_transaction_time)
    else:
        return None

In [129]:
# Group by person id

# Obtain total amount spent per person
person_transaction_df = transactions_df.groupby(["person"])["amount"].sum().reset_index(name="total_amt")
# Merge total number of transactions per person during the 1-month period (monthly frequency)
person_transaction_df = person_transaction_df.merge(
    transactions_df.groupby(["person"])["amount"].count().reset_index(name="monthly_freq"), how="left", on="person")

# Obtain list of transaction times for each person
person_transaction_times_df = transactions_df.groupby(["person"])["time"].apply(list).reset_index(name="txn_times")
# Obtain average time in between transactions
person_transaction_times_df["ave_btwn_time"] = [
    get_average_time_between_transactions(time_list) for time_list in person_transaction_times_df["txn_times"]]
# Obtain recency: end of period - last transaction time
person_transaction_times_df["recency"] = [
    get_hours_since_last_transaction(time_list) for time_list in person_transaction_times_df["txn_times"]]
# Merge with person transaction df
person_transaction_df = person_transaction_df.merge(person_transaction_times_df, how="left", on="person")

# Obtain number of unique weeks that a person has recorded transactions
transaction_week_count_df = transactions_df.groupby(["person"])["week"].nunique().reset_index(name="num_weeks")
# Obtain proportion of weeks with transactions compared to total number of weeks in period. 
transaction_week_count_df["prop_wks_w_txn"] = transaction_week_count_df["num_weeks"] / transactions_df["week"].nunique()
# Merge with person transaction df
transaction_week_count_df = transaction_week_count_df.drop(columns=["num_weeks"])
person_transaction_df = person_transaction_df.merge(transaction_week_count_df, how="left", on="person")

# Obtain average amount spent each transaction
person_transaction_df["ave_txn_amt"] = round((person_transaction_df["total_amt"] / person_transaction_df["monthly_freq"]), 2)

person_transaction_df.info()
person_transaction_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16578 entries, 0 to 16577
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   person          16578 non-null  object 
 1   total_amt       16578 non-null  float64
 2   monthly_freq    16578 non-null  int64  
 3   txn_times       16578 non-null  object 
 4   ave_btwn_time   16578 non-null  float64
 5   recency         16578 non-null  int64  
 6   prop_wks_w_txn  16578 non-null  float64
 7   ave_txn_amt     16578 non-null  float64
dtypes: float64(4), int64(2), object(2)
memory usage: 1.0+ MB


,person,total_amt,monthly_freq,txn_times,ave_btwn_time,recency,prop_wks_w_txn,ave_txn_amt
0,0009655768c64bdeb2e877511632db8f,127.60,8,"[228, 414, 528, 552, 576, 660, 690, 696]",66.86,18,0.8,15.95
1,00116118485d4dfda04fdbaba9a87b5c,4.09,3,"[294, 456, 474]",90.00,240,0.4,1.36
2,0011e0d4e6b944f998e987f904e8c1e5,79.46,5,"[132, 252, 576, 642, 654]",130.50,60,0.6,15.89
3,0020c2b971eb4e9188eac86d93036a77,196.86,8,"[54, 66, 72, 144, 510, 540, 696, 708]",93.43,6,0.6,24.61
4,0020ccbbb6d84e358d3414a3ff76cffd,154.05,12,"[42, 222, 240, 276, 300, 378, 426, 432, 450, 4...",57.27,42,1.0,12.84


In [130]:
# Merge with profile dataframe
customer_profile_df = profile_df.copy()[["id", "became_member_on", "gender", "age", "income"]].rename(columns={"id": "person"})
customer_profile_df = customer_profile_df.merge(person_transaction_df, how="left", on="person")

customer_profile_df.info()
customer_profile_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   person            17000 non-null  object        
 1   became_member_on  17000 non-null  datetime64[ns]
 2   gender            14825 non-null  object        
 3   age               17000 non-null  int64         
 4   income            14825 non-null  float64       
 5   total_amt         16578 non-null  float64       
 6   monthly_freq      16578 non-null  float64       
 7   txn_times         16578 non-null  object        
 8   ave_btwn_time     16578 non-null  float64       
 9   recency           16578 non-null  float64       
 10  prop_wks_w_txn    16578 non-null  float64       
 11  ave_txn_amt       16578 non-null  float64       
dtypes: datetime64[ns](1), float64(7), int64(1), object(3)
memory usage: 1.6+ MB


,person,became_member_on,gender,age,income,total_amt,monthly_freq,txn_times,ave_btwn_time,recency,prop_wks_w_txn,ave_txn_amt
0,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,118,NaN,20.40,9.0,"[360, 414, 444, 510, 534, 552, 606, 630, 696]",42.0,18.0,0.6,2.27
1,0610b486422d4921ae7d2bf64640c50b,2017-07-15,F,55,112000.0,77.01,3.0,"[18, 144, 528]",255.0,186.0,0.4,25.67
2,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN,118,NaN,14.30,6.0,"[132, 348, 450, 474, 636, 696]",112.8,18.0,0.8,2.38
3,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,F,75,100000.0,159.27,7.0,"[132, 144, 222, 240, 378, 510, 534]",67.0,180.0,0.8,22.75
4,a03223e636434f42ac4c3df47e8bac43,2017-08-04,NaN,118,NaN,4.65,3.0,"[234, 264, 612]",189.0,102.0,0.4,1.55


In [132]:
# Fill missing values

# Fill with 0
customer_profile_df[["total_amt", "monthly_freq", "prop_wks_w_txn", "ave_txn_amt"]] = customer_profile_df[
    ["total_amt", "monthly_freq", "prop_wks_w_txn", "ave_txn_amt"]].fillna(0.0)

# Fill with 715 (Max time in this dataset is 714 hours)
customer_profile_df[["ave_btwn_time", "recency"]] = customer_profile_df[["ave_btwn_time", "recency"]].fillna(715.0)

customer_profile_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   person            17000 non-null  object        
 1   became_member_on  17000 non-null  datetime64[ns]
 2   gender            14825 non-null  object        
 3   age               17000 non-null  int64         
 4   income            14825 non-null  float64       
 5   total_amt         17000 non-null  float64       
 6   monthly_freq      17000 non-null  float64       
 7   txn_times         16578 non-null  object        
 8   ave_btwn_time     17000 non-null  float64       
 9   recency           17000 non-null  float64       
 10  prop_wks_w_txn    17000 non-null  float64       
 11  ave_txn_amt       17000 non-null  float64       
dtypes: datetime64[ns](1), float64(7), int64(1), object(3)
memory usage: 1.6+ MB


In [ ]:
# Save to csv file
customer_profile_df.to_csv("customer_profile_df.csv")